<a href="https://colab.research.google.com/github/henriquehsilva/i2a2-challenges/blob/master/I2A2_Bone_Age_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
from IPython.display import HTML, display

# I2A2 - Bone Age Regression
---



## Contexto
> ### Criar um modelo para prever a idade óssea de crianças pelo raio-x da mão.


*   O conjunto de dados de treinamento vem da competição RSNA 2017
*   O conjunto de dados de teste é de um hospital brasileiro

**Diretivas primárias:** 


1. Generalizar o modelo para lidar com a mudança de distribuição entre os conjuntos de dados;
2. O conjunto de dados de **TREINAMENTO contém apenas imagens com a mão esquerda**, o conjunto de dados de **TESTE contém algumas imagens com as duas mãos**;
3. Ajustar o modelo, para lidar com a variãncia de resultados entre pacientes de sexos opostos;

---
## Visão geral

Uma das principais preocupações da medicina e da odontologia legal é a busca
de mecanismos que permitam a determinação da idade de pessoas e, para o desenvolvimento
deste trabalho, escolheu-se a **região de mão e pulso**, para a determinação da idade e do
desenvolvimento ósseo, devido a seqüência cronológica que a mesma apresenta bem como a
quantidade de ossos e epífises em uma área não muito extensa sendo possível realizar uma
única tomada radiográfica, evitando-se exposições desnecessárias ao paciente. 



In [65]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/Zs1oC_upTvM" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe> ' \
     '<iframe width="560" height="315" src="https://www.youtube.com/embed/WIk5S0BHotY" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe> ')


Após estudo da tese de mestrado do cirurgião dentista Mauricio Roberto Bosquiero: [DETERMINAÇÃO DA MATURIDADE ESQUELÉTICA
E ESTIMATIVA DA IDADE ATRA VÊS DE
RADIOGRAFIAS CARPAIS.](https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/docs/tese_mauricio_roberto.pdf), podemos chegar a uma terceira diretiva, referente a clara variância da maturidade esquelética de pacientes de mesma idade e sexos opostos.

---


> ### Pretendo avaliar o desempenho dos seguintes modelos pré-treinados: <br />
**ResNet-50**, **Inception-V3** e **VGG-16**, a fim de criar um desenho apropriado da camada pré-treinada para CNNs na previsão da idade óssea.

In [34]:
# Sweetviz: Python library for Exploratory data analysis (EDA)
!pip install sweetviz

ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [21]:
import tensorflow as tf
print(f"TensorFlow V{tf.__version__} 🦾")

TensorFlow V2.2.0 🦾


In [36]:
import pandas as pd
import sweetviz
import requests

from sklearn.model_selection import train_test_split
from google.colab import auth
from datetime import datetime


# Configurando conjuntos de desenvolvimento e de teste
---

O **Google Cloud Storage** foi escolhido como centralizador dos arquivos de suporte, bem como aos novos arquivos e relatórios gerados, por oferecer a disponibilidade e a capacidade necessárias para se trabalhar de forma distribuida.


In [23]:
GCP_STORAGE_BASE_URI = "https://storage.googleapis.com/bone-age-regression.henriquesilva.dev"

TRAIN_URI = f"{GCP_STORAGE_BASE_URI}/train.csv"
TEST_URI = f"{GCP_STORAGE_BASE_URI}/test.csv"

full_train_df = pd.read_csv(TRAIN_URI)
test = pd.read_csv(TEST_URI)

In [12]:
now = datetime.now()
timestamp = now.strftime("%Y%m%d%H%M%S")

timestamp = 20200714170254


In [24]:
full_train_df['boneage_category'] = pd.cut(full_train_df['boneage'], 10)

In [49]:
def image_link_generator(path_uri, file_name):
    image_link = f"{path_uri}/images/{file_name}"

    return image_link

print(image_link_generator(GCP_STORAGE_BASE_URI, '1377.png'))

full_train_df['image_link'] = full_train_df['fileName'].map(
    lambda file_name: image_link_generator(GCP_STORAGE_BASE_URI, file_name)
)

https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images/1377.png


In [50]:
HTML(full_train_df.head(5).to_html(render_links=True, escape=False))

,fileName,patientSex,boneage,boneage_category,image,image_link
0,1377.png,F,180,"(159.9, 182.6]",,https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images/1377.png
1,1378.png,F,12,"(0.773, 23.7]",,https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images/1378.png
2,1379.png,F,94,"(91.8, 114.5]",,https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images/1379.png
3,1380.png,M,120,"(114.5, 137.2]",,https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images/1380.png
4,1381.png,F,82,"(69.1, 91.8]",,https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/images/1381.png


In [ ]:

train_df, valid_df = train_test_split(
    full_train_df, 
    test_size = 0.25, 
    random_state = 2018,
    stratify = full_train_df['boneage_category']
  )

print('train', train_df.shape[0], 'validation', valid_df.shape[0])

train 9458 validation 3153


In [ ]:
raw_train_df.head()

,fileName,patientSex,boneage,boneagezscore,boneageCategory
2110,3726.png,F,162,162.0,"(159.9, 182.6]"
2066,3680.png,F,42,42.0,"(23.7, 46.4]"
420,1849.png,M,101,101.0,"(91.8, 114.5]"
11116,13950.png,M,156,156.0,"(137.2, 159.9]"
2995,4895.png,M,96,96.0,"(91.8, 114.5]"


In [14]:
def image_link_generator(path_uri, file_name):
  return f'<a href="{path_uri}/images/{file_name}><img src="{path_uri}/images/{file_name}" alt="{file_name}"" style="width:50px;height:50px;"></a>'

full_train_df['image'] = full_train_df['fileName'].map(
    lambda file_name: image_link_generator(GCP_STORAGE_BASE_URI, file_name)
)

NameError: ignored

In [ ]:
HTML(train.sample(10).to_html(render_links=True, escape=False))

# Exploratory data analysis (EDA)

# It is an essential early step in most data science projects and it often consists of taking the same steps to characterize a dataset (e.g. find out data types, missing information, distribution of values, correlations, etc.).

Running this command will perform the analysis and create the report object. To get the output, simply use the `show_html()` command:

In [ ]:
valid = valid_df.drop(columns=['fileName', 'boneage_category'])

In [ ]:
train = train_df.drop(columns=['fileName', 'boneage_category'])

In [ ]:
print(time)

NameError: ignored

In [ ]:
report_file = f"{timestamp}_report_view.html"

data_report = sweetviz.compare([train, "Train"], [valid, "Valid"], "boneage")
data_report.show_html(report_file)





                                   |                         | [  0%]   00:00  -> (? left)



Summarizing dataframe:             |                         | [  0%]   00:00  -> (? left)



:TARGET::                          |███████              | [ 33%]   00:00  -> (00:00 left)



:TARGET::                          |██████████████       | [ 67%]   00:00  -> (00:00 left)



:patientSex:                       |██████████████       | [ 67%]   00:00  -> (00:00 left)



:patientSex:                       |█████████████████████| [100%]   00:01  -> (00:00 left)



:FEATURES DONE:                    |█████████████████████| [100%]   00:01  -> (00:00 left)




                                   |                         | [  0%]   00:00  -> (? left)



:Processing Pairwise Features:     |                         | [  0%]   00:00  -> (? left)



:PAIRWISE DONE:                    |█████████████████████| [100%]   00:00  -> (00:00 left)


Creating Associations graph... DONE!


In [ ]:
auth.authenticate_user()

GCP_PROJECT_ID = 'i2a2-283020'
GCP_BUCKET_NAME = 'bone-age-regression.henriquesilva.dev'

!gcloud config set project {GCP_PROJECT_ID}

!gsutil cp /content/"{report_file}" gs://{GCP_BUCKET_NAME}/reports/


Updated property [core/project].
Copying file:///content/20200714002253_report_view.html [Content-Type=text/html]...
-
Operation completed over 1 objects/553.7 KiB.                                    


In [ ]:
train['boneage_category'] = pd.cut(train['boneage'], 10)
train.sample(3)

,patientSex,boneage,boneage_category
11525,F,120,"(114.5, 137.2]"
10189,F,132,"(114.5, 137.2]"
5036,F,9,"(0.773, 23.7]"


In [ ]:
response = requests.get(f"{GCP_STORAGE_BASE_URI}/reports/{report_file}")

if (response.status_code == 200):
  print(f"Open your report it using the default browser like link: {response.url}")
else:
  print("Sorry, the report not found.")

Open your report it using the default browser like link: https://storage.googleapis.com/bone-age-regression.henriquesilva.dev/reports/20200714002253_report_view.html
